In [1]:
# !pip install llama-index==0.8.9

In [1]:
import os
os.environ["LANGCHAIN_TELEMETRY"] = "false"

from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import chromadb
from sqlalchemy.orm.collections import collection
from langchain.text_splitter import RecursiveCharacterTextSplitter

/Users/imbilalbutt/PycharmProjects/RAGpipelineChatbotwithFastAPI/rag-env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/imbilalbutt/PycharmProjects/RAGpipelineChatbotwithFastAPI/rag-env/lib/python3.9/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# ---------------------------
# 1. Load PDF documents
# ---------------------------
docs_dir = "docs/"
pdf_files = [os.path.join(docs_dir, f) for f in os.listdir(docs_dir) if f.endswith(".pdf")][:10]

documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pages = loader.load()
    documents.extend(pages)

print(f"Loaded {len(documents)} pages from {len(pdf_files)} PDFs.")


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)


Loaded 284 pages from 10 PDFs.


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(documents)

In [22]:
# chunks

In [5]:
# ---------------------------
# 2. Create embeddings
# ---------------------------
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


/var/folders/sb/ks236ylx60gdnvp1m0q6wbpr0000gn/T/ipykernel_7189/1078671494.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [24]:
# embeddings

In [6]:
# # ---------------------------
# # 3. Store embeddings in Chroma
# # ---------------------------
# from langchain_chroma import Chroma
#
# vector_db_dir = "./chroma_db"
# if not os.path.exists(vector_db_dir):
#     os.makedirs(vector_db_dir)
#
# db = Chroma.from_documents(chunks, embeddings, persist_directory=vector_db_dir)
# db.persist()
# print("Embeddings stored in Chroma vector database.")


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [10]:
from langchain_core.vectorstores import InMemoryVectorStore
db = InMemoryVectorStore(embeddings)

In [33]:
# from langchain_chroma import Chroma
# vector_db_dir = "./chroma_db"
# db = Chroma(collection_name = "my_collection", embedding_function = embeddings, persist_directory = vector_db_dir)

In [11]:
# ---------------------------
# 4. Create LangChain retriever
# ---------------------------
retriever = db.as_retriever()


In [12]:
# ---------------------------
# 5. Create RetrievalQA chain with LLM
# ---------------------------
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(model="gpt-3.5-turbo"),
    chain_type="stuff",
    retriever=retriever
)


/var/folders/sb/ks236ylx60gdnvp1m0q6wbpr0000gn/T/ipykernel_7189/166887468.py:5: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm=OpenAI(model="gpt-3.5-turbo"),


In [25]:
# ---------------------------
# 6. Create LlamaIndex Index
# ---------------------------
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.core.text_splitter import TokenTextSplitter, SentenceSplitter
from llama_index.llms.openai import OpenAI

# Set LLM globally for LlamaIndex
Settings.llm = OpenAI(model="gpt-4o-mini")  # or "gpt-4o", "gpt-3.5-turbo", etc.

# Convert LangChain Documents → LlamaIndex Documents
li_documents = [
    Document(text=doc.page_content, metadata=doc.metadata)
    for doc in documents
]

# Create sentence splitter
sentence_splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=200
)

nodes = sentence_splitter.get_nodes_from_documents(li_documents)

# Build Vector Index from Nodes (not from_documents!)
index = VectorStoreIndex(nodes=nodes)

print(f"LlamaIndex vector index created successfully with {len(nodes)} nodes.")



RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [18]:
# ---------------------------
# 7. Ask questions
# ---------------------------
while True:
    query = input("\nEnter your question (or 'exit' to quit): ")
    if query.lower() == "exit":
        break

    # Retrieve answer using LangChain RAG
    answer = qa_chain.run(query)
    print("\nAnswer (LangChain RAG):")
    print(answer)

    # also query LlamaIndex
    response = index.as_query_engine().query(query)
    print("\nAnswer (LlamaIndex):")
    print(response)

/var/folders/sb/ks236ylx60gdnvp1m0q6wbpr0000gn/T/ipykernel_7189/2053520545.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}